# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [13]:
import pandas as pd
import requests

# 1) Carregar o arquivo
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Mostrar o número total de registros (linhas, colunas)
print("Formato original (linhas, colunas):", sinasc.shape)

# Verificar o número de registros únicos (sem duplicatas)
registros_unicos = sinasc.drop_duplicates().shape
print("Formato sem duplicatas:", registros_unicos)

# Conclusão
if sinasc.shape[0] == registros_unicos[0]:
    print("Não há duplicatas.")
else:
    print("Existem duplicatas.")

Formato original (linhas, colunas): (27028, 69)
Formato sem duplicatas: (27028, 69)
Não há duplicatas.


In [14]:
# 2) Contar o número de valores missing (NaN) por variável
missing_por_coluna = sinasc.isnull().sum().sort_values(ascending=False)

# Exibir apenas as variáveis que têm pelo menos 1 valor ausente
print(missing_por_coluna[missing_por_coluna > 0])


DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
CODOCUPMAE     2907
MESPRENAT      2867
QTDFILMORT     2098
QTDPARTNOR     1879
QTDPARTCES     1791
QTDFILVIVO     1573
TPMETESTIM     1238
SEMAGESTAC     1238
GESTACAO       1232
QTDGESTANT     1212
STTRABPART      947
CONSPRENAT      930
STCESPARTO      747
RACACORMAE      661
RACACOR         647
IDANOMAL        591
ESTCIVMAE       317
ESCMAE          312
CODMUNNATU      298
NATURALMAE      298
CODUFNATU       298
TPAPRESENT      265
ESCMAE2010      249
ESCMAEAGR1      249
CODESTAB        115
APGAR5          103
APGAR1           96
GRAVIDEZ         79
TPFUNCRESP       67
TPNASCASSI       61
DTDECLARAC       52
PARTO            49
DTNASCMAE        40
HORANASC         21
TPDOCRESP        14
SEXO              4
munResLon         1
munResAlt         1
munResLat         1
munResArea        1
dtype: int64


In [15]:
colunas_interesse = [
    'LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'
] 

# 3) Selecionar apenas as colunas de interesse
sinasc_reduzido = sinasc[colunas_interesse].copy()

# Contar valores missing nessas colunas
missing_reduzido = sinasc_reduzido.isnull().sum().sort_values(ascending=False)
print(missing_reduzido[missing_reduzido > 0])

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
dtype: int64


In [16]:
# 4) Remover registros com APGAR5 ausente
sinasc_apgar = sinasc_reduzido.dropna(subset=['APGAR5'])

# Contar número de linhas após remoção
print("Número de linhas após remover APGAR5 ausente:", len(sinasc_apgar))

# Verificar se ainda há valores missing
print("Valores missing restantes:")
print(sinasc_apgar.isnull().sum().sort_values(ascending=False))

Número de linhas após remover APGAR5 ausente: 26925
Valores missing restantes:
QTDFILVIVO    1566
GESTACAO      1216
ESTCIVMAE      315
ESCMAE         310
GRAVIDEZ        76
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
APGAR5           0
dtype: int64


In [17]:
# 5) Substituir NaN por 9 em ESTCIVMAE e CONSULTAS, sem warning
sinasc_apgar.loc[:, 'ESTCIVMAE'] = sinasc_apgar['ESTCIVMAE'].fillna(9)
sinasc_apgar.loc[:, 'CONSULTAS'] = sinasc_apgar['CONSULTAS'].fillna(9)

# Verificar se deu certo
print(sinasc_apgar[['ESTCIVMAE', 'CONSULTAS']].isnull().sum())


ESTCIVMAE    0
CONSULTAS    0
dtype: int64


In [18]:
# 6) Substituir valores ausentes em QTDFILVIVO por 0
sinasc_apgar.loc[:, 'QTDFILVIVO'] = sinasc_apgar['QTDFILVIVO'].fillna(0)

# Verificar se ainda há valores ausentes nessa coluna
print("Missing em QTDFILVIVO:", sinasc_apgar['QTDFILVIVO'].isnull().sum())


Missing em QTDFILVIVO: 0


In [20]:
# 7)  Verificar variáveis restantes com valores ausentes
missing_restantes = sinasc_apgar.isnull().sum()
print(missing_restantes[missing_restantes > 0])

# Preencher ESCMAE com 9 (ignorado)
sinasc_apgar.loc[:, 'ESCMAE'] = sinasc_apgar['ESCMAE'].fillna(9)

# Preencher GESTACAO com 9 (ignorado)
sinasc_apgar.loc[:, 'GESTACAO'] = sinasc_apgar['GESTACAO'].fillna(9)

# Preencher GRAVIDEZ com 1 (valor mais provável)
sinasc_apgar.loc[:, 'GRAVIDEZ'] = sinasc_apgar['GRAVIDEZ'].fillna(1)

# Verificar se ainda há valores ausentes
print(sinasc_apgar.isnull().sum().sort_values(ascending=False))


Series([], dtype: int64)
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [22]:
# 8) Criar uma cópia explícita para evitar warnings
sinasc_apgar = sinasc_apgar.copy()

# Função de categorização de Apgar5
def classificar_apgar(valor):
    if valor >= 8:
        return 'normal'
    elif valor >= 6:
        return 'asfixia leve'
    elif valor >= 4:
        return 'asfixia moderada'
    else:
        return 'asfixia severa'

# Aplicar a função e salvar a nova variável
sinasc_apgar['APGAR5_CATEGORIA'] = sinasc_apgar['APGAR5'].apply(classificar_apgar)

# Mostrar frequência das categorias
print(sinasc_apgar['APGAR5_CATEGORIA'].value_counts())

APGAR5_CATEGORIA
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [23]:
# 9) Renomear colunas para snake_case
sinasc_apgar.rename(
    columns=lambda col: col.strip().lower().replace(" ", "_"), 
    inplace=True
)

# Visualizar as novas colunas
print(sinasc_apgar.columns.tolist())


['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_categoria']
